In [2]:
import numpy
import os
import pandas

# import scipy.special
import  iqplot

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [54]:
CSV_FILE_PATH = os.path.join("data", "grant_complete.csv")
COLUMN_OF_INTEREST = "beak length (mm)"
YEARS = [1975, 2012]

df = pandas.read_csv(CSV_FILE_PATH, comment="#")

In [55]:
species_mask = df["species"] == "scandens"
year_mask = df["year"].isin(YEARS)

df = df.loc[(species_mask & year_mask), ["year", COLUMN_OF_INTEREST]]

In [56]:
figure = iqplot.ecdf(
    data=df,
    q=COLUMN_OF_INTEREST,
    cats="year"
)

bokeh.io.show(figure)

In [57]:
values_by_year = {
    year: df.loc[df["year"] == year, COLUMN_OF_INTEREST].values
    
    for year in df["year"].unique()
}

In [58]:
[numpy.mean(values_by_year[year]) for year in values_by_year] 

[14.120919540229885, 13.428333333333333]

In [59]:
NUM_REPETITIONS = 10000

bootstrap_replicates = {
    year: numpy.empty(NUM_REPETITIONS)
    for year in values_by_year
}

random = numpy.random.default_rng(42)

for i in range(NUM_REPETITIONS):
    
    for year, values in values_by_year.items():
        
        bootstrap_samples = random.choice(
            values,
            size=len(values),
            replace=True
        )
        
        bootstrap_replicates[year][i] = numpy.mean(bootstrap_samples)

confidence_intervals = {
    year: numpy.percentile(bootstrap_replicates[year], [2.5, 97.5])
    for year in bootstrap_replicates
}

In [60]:
confidence_intervals

{1975: array([13.96666379, 14.27597701]),
 2012: array([13.30055556, 13.55279563])}

In [61]:
from scipy import stats

In [62]:
figure = iqplot.ecdf(
    data=bootstrap_replicates[1975]
)

bokeh.io.show(figure)

In [63]:
sem_confidence_intervals = {
    year: (
        values_by_year[year].mean() - 1.96*stats.sem(values_by_year[year]),
        values_by_year[year].mean() + 1.96*stats.sem(values_by_year[year])
    )
    for year in values_by_year
}

In [64]:
sem_confidence_intervals

{1975: (13.96231270920305, 14.279526371256718),
 2012: (13.302871217227404, 13.553795449439262)}

In [65]:
year_strings = [str(year) for year in YEARS]

figure = bokeh.plotting.figure(
    frame_height=100,
    frame_width=200,
    x_axis_label=COLUMN_OF_INTEREST,
    y_range=year_strings
)

figure.circle(
    [values_by_year[year].mean() for year in YEARS],
    year_strings,
    size=5
)

for year in YEARS:
    figure.line(
        confidence_intervals[year],
        [str(year)]*2,
        line_width=3
    )

bokeh.io.show(figure)

In [66]:
numpy.percentile(
    bootstrap_replicates[2012] - bootstrap_replicates[1975],
    [2.5, 97.5]
)

array([-0.89187479, -0.49802942])